This notebook is an example of a single run of simulation. It generates a species pool of size 24, contains species with different growth rates and diauxic preference orders. These species invade one by one in a system under serial dilution, and the community is assembled in this process. 

See the paper for more details. 

In [1]:
exec(open("ModulesWithLags.py", encoding = 'utf-8').read())

In [2]:
Nr = 4
theta = 1e-1 # hard minimum of growth rates
g_mu  = 0.25
g_sigma= 0.05
Size = 1
lagTime = 1
# dynamic parameters
D = 0.01 # (inverse) dilution factor
T_dilute = 24 # time interval between dilutions
dilute_to_steady = 10 # #(dilution) between invasions
b0 = 1e-6 # density of bug when introduced/initial
b_threshold = 1e-6 # extinction density
c_threshold = 1e-9 # concentration threshold

Nb = factorial(Nr)*Size
yields_list = 0.5*np.ones([Nb, Nr])
growth_rate_list  = []
for i in range(Nb):
    growth_rate_list.append([max(np.random.normal(g_mu, g_sigma), theta) for i in range(Nr)])
invlist = list(range(Nb))
random.shuffle(invlist)
details = {'res_begin':[], 't_info':[], 'res_left':[], 'bug_info':[], 'round_idx':[0]}
Res = [1.0 for i in range(Nr)]
preference_list = list(itertools.permutations(range(Nr), Nr))
start=time.time()
movestep=0
details = {'res_begin':[], 't_info':[], 'res_left':[], 'bug_info':[], 'round_idx':[0]}
system = {'res_available': np.heaviside(Res, 0), 'res_concentration': [i for i in Res], 'bug_available': [0 for i in range(Nb)], 'bug_density': [0 for i in range(Nb)]}
old_ext_list = invlist
system, new_ext_list = round_robin_invade(system, old_ext_list, growth_rate_list, lagTime)
new_ext_list = [i for i in invlist if i in new_ext_list]
count = 0
cycled = False
while new_ext_list != old_ext_list and count < 10:
    old_ext_list = new_ext_list
    survivors, concent, pref_list, growth = output(system)
    system, new_ext_list = round_robin_invade(system, old_ext_list, growth_rate_list, lagTime)
    count = count + 1
    if count > 10:
        cycled = True
        print('A cycle might have occured')
details['round_idx'] = details['round_idx'][1:]
end = time.time()
print('survivors are:' + str(output(system)[0]))
print('Their abundances:' + str(output(system)[1]))
print('Their preference orders:'+ str(output(system)[2]))
#print(end-start)

survivors are:[2, 11, 18]
Their abundances:[0.010001000100011138, 0.004428754041211744, 0.00557224605879826]
Their preference orders:[(0, 2, 1, 3), (1, 3, 2, 0), (3, 0, 1, 2)]
